# 🗳️ Eurovision Voting Analysis
This notebook analyses the Eurovision voting data from 2007 to 2024: how votes were distributed, received, and reciprocated among participating countries.

### 📥 Load and Filter Grand Final Voting Data

In [1]:
# Load voting data
import pandas as pd

votes_data = pd.read_csv("vote_since_2007.csv")
votes_data = votes_data.loc[votes_data['round'] == 'final'] #filter for the 'final' round only
votes_data

,year,round,from_country_id,to_country_id,total_points
0,2007,final,al,rs,1.0
1,2007,final,al,ua,0.0
2,2007,final,al,ru,0.0
3,2007,final,al,tr,10.0
4,2007,final,al,bg,0.0
...,...,...,...,...,...
28872,2024,final,at,no,1.0
28873,2024,final,cz,no,1.0
28874,2024,final,fi,no,3.0
28875,2024,final,lt,no,2.0


### 📊 Votes Received
We compute **vote ratio** = (number of countries who voted for a country) / (total number of participating countries - 1)


In [18]:
total_participating_countries = votes_data.groupby(['year', 'round']).agg(
    total_countries=('from_country_id', 'nunique')
).reset_index()

#account for the fact that a country cannot vote for itself
total_participating_countries['total_countries-1'] = total_participating_countries['total_countries'] - 1
total_participating_countries

,year,round,total_countries,total_countries-1
0,2007,final,42,41
1,2008,final,43,42
2,2009,final,42,41
3,2010,final,39,38
4,2011,final,43,42
5,2012,final,42,41
6,2013,final,39,38
7,2014,final,37,36
8,2015,final,40,39
9,2016,final,42,41


In [19]:
# Filtering out entries where total_points is zero, as they do not count as a vote
non_zero_votes = votes_data[votes_data['total_points'] > 0]

# Grouping the data by year, round, and to_country_id to count distinct from_country_id that voted
voting_summary = non_zero_votes.groupby(['year', 'round', 'to_country_id']).agg(
    number_of_votes=('from_country_id', 'nunique')
).reset_index()
voting_summary

,year,round,to_country_id,number_of_votes
0,2007,final,am,18
1,2007,final,ba,17
2,2007,final,bg,27
3,2007,final,by,24
4,2007,final,de,12
...,...,...,...,...
427,2024,final,pt,25
428,2024,final,rs,11
429,2024,final,se,28
430,2024,final,si,4


In [20]:
country_names = {
    'gr': 'Greece',
    'no': 'Norway',
    'se': 'Sweden',
    'az': 'Azerbaijan',
    'md': 'Moldova',
    'ua': 'Ukraine',
    'is': 'Iceland',
    'lt': 'Lithuania',
    'rs': 'Serbia',
    'al': 'Albania',
    'ro': 'Romania',
    'dk': 'Denmark',
    'ee': 'Estonia',
    'fi': 'Finland',
    'il': 'Israel',
    'be': 'Belgium',
    'ru': 'Russia',
    'cy': 'Cyprus',
    'am': 'Armenia',
    'si': 'Slovenia',
    'nl': 'Netherlands',
    'mt': 'Malta',
    'hu': 'Hungary',
    'ch': 'Switzerland',
    'ge': 'Georgia',
    'pt': 'Portugal',
    'pl': 'Poland',
    'ie': 'Ireland',
    'lv': 'Latvia',
    'by': 'Belarus',
    'mk': 'Macedonia',
    'hr': 'Croatia',
    'at': 'Austria',
    'bg': 'Bulgaria',
    'fr': 'France',
    'es': 'Spain',
    'gb': 'United Kingdom',
    'de': 'Germany',
    'sm': 'San Marino',
    'cz': 'Czech Republic',
    'au': 'Australia',
    'ba': 'Bosnia and Herzegovina',
    'ad': 'Andorra',
    'tr': 'Turkey',
    'it': 'Italy',
    'sk': 'Slovakia',
    'me': 'Montenegro'            
}

# Applying the mapping to the dataset
voting_summary['to_country_name'] = voting_summary['to_country_id'].map(country_names)
voting_summary

,year,round,to_country_id,number_of_votes,to_country_name
0,2007,final,am,18,Armenia
1,2007,final,ba,17,Bosnia and Herzegovina
2,2007,final,bg,27,Bulgaria
3,2007,final,by,24,Belarus
4,2007,final,de,12,Germany
...,...,...,...,...,...
427,2024,final,pt,25,Portugal
428,2024,final,rs,11,Serbia
429,2024,final,se,28,Sweden
430,2024,final,si,4,Slovenia


In [21]:
voting_ratio_summary = pd.merge(voting_summary, total_participating_countries, on=['year', 'round'])
voting_ratio_summary['vote_ratio'] = voting_ratio_summary['number_of_votes'] / voting_ratio_summary['total_countries-1']
voting_ratio_summary

,year,round,to_country_id,number_of_votes,to_country_name,total_countries,total_countries-1,vote_ratio
0,2007,final,am,18,Armenia,42,41,0.439024
1,2007,final,ba,17,Bosnia and Herzegovina,42,41,0.414634
2,2007,final,bg,27,Bulgaria,42,41,0.658537
3,2007,final,by,24,Belarus,42,41,0.585366
4,2007,final,de,12,Germany,42,41,0.292683
...,...,...,...,...,...,...,...,...
427,2024,final,pt,25,Portugal,38,37,0.675676
428,2024,final,rs,11,Serbia,38,37,0.297297
429,2024,final,se,28,Sweden,38,37,0.756757
430,2024,final,si,4,Slovenia,38,37,0.108108


In [22]:
voting_ratio_summary.to_csv('final_votes.csv')

### 📊 Vote Reciprocation

Vote reciprocation in Eurovision refers to the mutual exchange of points between two countries. If Country A gives points to Country B and Country B also gives points to Country A. It can be calculated by counting such bidirectional voting pairs and comparing them to the total number of possible voting interactions.

We compute vote reciprocaion = (x votes reciprocated) / (n possible votes)

In [23]:
# First, calculate the total votes received for each country per year and round
total_votes_received = non_zero_votes.groupby(['year', 'round', 'to_country_id']).size().reset_index(name='total_votes_received')

# Merging the positive votes data frame with itself to find reciprocations
reciprocal_check = pd.merge(
    non_zero_votes, 
    non_zero_votes, 
    how='inner', 
    left_on=['year', 'round', 'from_country_id', 'to_country_id'], 
    right_on=['year', 'round', 'to_country_id', 'from_country_id']
)

# Filtering out self-votes (should not be possible in this context)
reciprocal_check = reciprocal_check[reciprocal_check['from_country_id_x'] != reciprocal_check['to_country_id_x']]
reciprocal_votes_count = reciprocal_check.groupby(['year', 'round', 'to_country_id_x']).size().reset_index(name='total_reciprocal_votes').rename(columns={'to_country_id_x': 'to_country_id'})

# Merging total_votes_received with total_reciprocal_votes
reciprocation_data = pd.merge(
    total_votes_received, 
    reciprocal_votes_count, 
    on=['year', 'round', 'to_country_id'],
    how='left'
)

# Fill missing values in total_reciprocal_votes with 0 (countries that received no reciprocated votes)
reciprocation_data['total_reciprocal_votes'].fillna(0, inplace=True)

reciprocation_data['to_country_name'] = reciprocation_data['to_country_id'].map(country_names)


# Calculating the reciprocation ratio
reciprocation_data['available_votes'] = reciprocation_data['year'].apply(lambda x: 10 if 2007 <= x <= 2015 else 20)
reciprocation_data['reciprocation_ratio'] = reciprocation_data['total_reciprocal_votes'] / reciprocation_data['available_votes']


reciprocation_data


/var/folders/nq/kj4xhqkj6nzdw_r4b9pphgnh0000gn/T/ipykernel_13831/649071085.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  reciprocation_data['total_reciprocal_votes'].fillna(0, inplace=True)


,year,round,to_country_id,total_votes_received,total_reciprocal_votes,to_country_name,available_votes,reciprocation_ratio
0,2007,final,am,18,8.0,Armenia,10,0.80
1,2007,final,ba,17,4.0,Bosnia and Herzegovina,10,0.40
2,2007,final,bg,27,9.0,Bulgaria,10,0.90
3,2007,final,by,24,8.0,Belarus,10,0.80
4,2007,final,de,12,1.0,Germany,10,0.10
...,...,...,...,...,...,...,...,...
427,2024,final,pt,25,11.0,Portugal,20,0.55
428,2024,final,rs,11,5.0,Serbia,20,0.25
429,2024,final,se,28,10.0,Sweden,20,0.50
430,2024,final,si,4,2.0,Slovenia,20,0.10


In [24]:
reciprocation_data.to_csv('final_reciprocation.csv')